In [3]:
#import libraries for PolyTrend algorithm and downloading the time series data
import pandas as pd
import ee
import numpy as np
import numpy.linalg as lng
import numpy.polynomial.polynomial as poly
import scipy.stats as stats
from ipyleaflet import Map, basemaps, DrawControl, basemap_to_tiles, CircleMarker
    
ee.Initialize()

<h3>To use your own data, upload a csv file to Jupyter notebook, enter the name of the file in the next cell and run the code. Time series needs to have at least 4 time steps to perform cubic fitting. First row should contain names of columns, including 'latitude' and 'longitude'. </h3>

In [ ]:
file_name = 'enter the name of you file here, as string, ending with .csv'
dataset= pd.read_csv(file_name)
dataset.groupby(['longitude', 'latitude'])
#check the records are really organised by lon/lat and if data looks fine
print(dataset.head())


<h3>Enter parameters:</h3>
<br>
<ul>
    <li>statistical significance (alpha), the default value is 0.05</li> 
    <li>coordinates of the region of interest. If you don't know them use the map, mark a polygon. The coordinate of the last polygon drawn will be used</li>
    <li>decide on the dataset you'd like to use. Check their IDs <a href="https://developers.google.com/earth-engine/datasets/catalog/">here</a>. Enter as variable name_of_collection</li>
    <li>you will filter the collection according to available dates and your interest. Enter start and end dates as strings</li>
    <li>enter name of the band you want to analyse. This is case sensitive so please check what band names the dataset has on Earth Engine website</li>
    <li>enter range in which your values should be, eg. for NDVI it would be 0 to 1</li>
</ul>

<h3>Variables entered by the user. Change the right side of the equation keeping the same format and type of data as in the example</h3> 

In [16]:
alpha = 0.05
name_of_collection = 'MODIS/006/MOD13A2'
start_date = '2000-12-01'
end_date = '2008-12-02'
band_name = 'NDVI'
# coords = [[11.944024, 52.30512], [11.922045, 52.197507], [11.966003, 52.224435], [11.944024, 52.30512]]
range_of_ndvi = range(0,1) #???a float? will be used for validation 


<h3>If you do not have coordinates of the area of interest create a map with the script below and mark a polygon</h3>

In [6]:
m = Map(
    center=(52.834502, 13.798697),
    zoom=7
)
#enable drawing tools
draw_control = DrawControl()
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 1.0
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False
}

m.add_control(draw_control)
m


Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…

<h3>Upload image collection and generate time series. To use coordinates of the polygon drawn on the above map. The last polygone drawn will be used</h3><br>
<i>Please note if you are using a point, you have to change the geometry inside the script so that 
    AOI = ee.Geometry.Point(coords)</i>

In [17]:
print('coordinates: ', draw_control.last_draw['geometry']['coordinates'])
coords = draw_control.last_draw['geometry']['coordinates']

collection = ee.ImageCollection(name_of_collection).filterDate(start_date, end_date).select(band_name)
crs = collection.first().getInfo()['bands'][0]['crs']
def GetDataFrame(coords):
    
    AOI = ee.Geometry.Polygon(coords)
    geom_values = collection.filterBounds(AOI).getRegion(geometry=AOI, scale=500, crs=crs)
    geom_values_list = ee.List(geom_values).getInfo()
    # Convert to a Pandas DataFrame.
    header = geom_values_list[0]
    data = pd.DataFrame(geom_values_list[1:], columns=header)
    data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
    data.set_index('time')
    data.groupby(['longitude', 'latitude'])
    #see how many images there are:
    return data;

try:
    dataset = GetDataFrame(coords)
    print('Dataset ready')
except:
    print('Something went wrong')

coordinates:  [[[11.059472, 52.725248], [11.059472, 52.618658], [11.290149, 52.638663], [11.059472, 52.725248]]]
Dataset ready


<h3>To save the time series for the polygon run the next line of code. It saves to the active Anaconda environment:</h3>

In [ ]:
dataset.to_csv('time_series.csv')

<h3>Below is the PolyTrend algorithm. Run it here so that it can be used in the next cell with the data</h3>

In [9]:
def PolyTrend(Y, alpha):    
    X = range(1, len(Y)+1)
    
    #define function to find p value:
    def Pvalue(coef, df, A, Aprim, pn):
        #generate square residual
        part_res = np.dot(A, pn)-Y
        residual = np.dot(part_res.transpose(), part_res)
        #generate variance-covariance matrix
        VC = lng.inv(np.dot(Aprim, A))*residual/df
        #compute variance of the first coefficient
        VC1 = np.sqrt(VC[0,0])
        #compute t-statistic
        statistic = coef/VC1
        #compute p value
        p = stats.t.sf(np.abs(statistic), df)*2
        return p;

    def Plinear(X, Y):
        degree = 1
        df1 = len(X)-degree-1
        #generate Vandermonde matrix
        A1 = np.vander(X, 2)
        #generate transpose of the Vandermonde matrix
        Aprim1 = A1.transpose()
        p1 = np.dot(np.dot((lng.inv(np.dot(Aprim1, A1))), Aprim1), Y)
        coef1 = p1[0]
        Plin = Pvalue(coef1, df1, A1, Aprim1, p1)
        Slope = p1[0]
        Direction = np.sign(Slope)
        return Plin;
    
    degree = 3
    #degrees of freedom
    df3 = len(X)-degree-1
    #generate Vandermonde matrix
    A3 = np.vander(X, 4)
    #generate transpose of the Vandermonde matrix
    Aprim3 = A3.transpose()
    #X=inv(A'*A)*A'*L - creating coefficients matrix:
    p3 = np.dot(np.dot((lng.inv(np.dot(Aprim3, A3))), Aprim3), Y)
    coef3 = p3[0]
    #compute p-value for cubic fit
    Pcubic = Pvalue(coef3, df3, A3, Aprim3, p3)
    #get roots of cubic polynomial
    coefs3 = ([p3[2], 2*p3[1], 3*p3[0]])
    roots3 = np.sort(poly.polyroots(coefs3))

    if (np.imag(roots3[0]) == 0 and np.imag(roots3[1])==0 and roots3[0] != roots3[1] and X[0] <= roots3[0] and roots3[0] <= len(X) and X[0] <= roots3[1] and roots3[1] <= len(X) and Pcubic < alpha):
        Plin = Plinear(X, Y)
        if (Plin < alpha):
            Trend_type = 3
            Significance = 1
            Poly_degree = 3
        else:
            Trend_type = -1
            Significance = -1
            Poly_degree = 3
            return [Trend_type, Significance, Poly_degree];
    else:
        degree = 2
        df2 = len(X)-degree-1
        A2 = np.vander(X, 3)
        Aprim2 = A2.transpose()
        p2 = np.dot(np.dot((lng.inv(np.dot(Aprim2, A2))), Aprim2), Y)
        coef2 = p2[0]
        Pquadratic = Pvalue(coef2, df2, A2, Aprim2, p2)
        coefs2 = ([p2[1], 2*p2[0]])
        roots2 = np.sort(poly.polyroots(coefs2))
        
        if (1<=roots2 and roots2 <= len(X) and Pquadratic < alpha):
            Plin = Plinear(X, Y)
            if Plin < alpha:
                Trend_type = 2
                Significance = 1
                Poly_degree = 2
            else:
                Trend_type = -1
                Significance = -1
                Poly_degree = 2
                return [Trend_type, Significance, Poly_degree];
                
        else:
            Plin = Plinear(X, Y)
            if Plin < alpha:
                Trend_type = 1
                Significance = 1
                Poly_degree = 1
            else:
                Trend_type = 0
                Significance = -1
                Poly_degree = 0
            return [Trend_type, Significance, Poly_degree];
        return [Trend_type, Significance, Poly_degree];
    return;

In [18]:
#establish how many images there are in the collection - it's our n
list_of_images = dataset['id']
ids_of_images = []
for img_id in list_of_images:
    if img_id not in ids_of_images:
        ids_of_images.append(img_id)
        
n = len(ids_of_images)
print('number of images: ', n)
number_of_pixels = len(dataset) 
print('number of pixels analysed: ', number_of_pixels)

#make_Y function returns coordinates for each pixel with the results of the analysis for each individual pixel
def make_Y(dataset, alpha):
    PT_result = []
    #split the dataset into pixel time series
    for i in range(0, number_of_pixels, n):
        Y = dataset[i:i+n][band_name].values 
        #check for NANs and infitite values in each vector
        if True in np.isnan(Y) or True in np.isinf(Y):
            result = ['NA', 'NA', 'NA']
        else:
            result = list(PolyTrend(Y, alpha))
        #populate the empty PT_result list with values    
        pixel_long = dataset.at[i, 'longitude']
        pixel_lat = dataset.at[i, 'latitude']
        PT_result_header = ['longitude', 'latitude', 'trend type', 'significance', 'degree']
        PT_result.append([pixel_long, pixel_lat, result[0], result[1], result[2]])
    #create a frame for displaying results on a map    
    image_frame = pd.DataFrame(PT_result[0:], columns=PT_result_header)
    return image_frame;

final_result = make_Y(dataset, alpha)
print('pixels to display: ', len(final_result))
print('Running this process was ended successfully. You can download results and display them on maps')
# final_result.to_csv('Fri_Result1.csv')

AOI_zoom = (final_result.at[0, 'latitude'], final_result.at[0, 'longitude'])

number of images:  184
number of pixels analysed:  67528


C:\Users\Admin\Anaconda3\envs\polytrend\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in sqrt
  if sys.path[0] == '':
C:\Users\Admin\Anaconda3\envs\polytrend\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\Admin\Anaconda3\envs\polytrend\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\Admin\Anaconda3\envs\polytrend\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


pixels to display:  367
Running this process was ended successfully. You can download results and display them on maps


<h3>Save results to a csv file in this Anaconda environment.</h3>

In [13]:
final_result.to_csv('PolyTrend_result.csv')

<h3>Run functions related to map generation</h3>

In [11]:
def assign_color(value):
    if value==-1:
        return 'yellow'
    elif value ==0:
        return 'red'
    elif value ==1:
        return 'orange'
    elif value == 2:
        return 'green'
    elif value == 3:
        return 'white'

<h2>Trend type graphic</h2>

In [19]:
m_trend = Map(
    center=AOI_zoom,
    zoom=13
)

# for i in range(0, 5):
for i in range(0, len(final_result)):  
    def assign_color(value):
        if value==-1:
            return 'yellow'
        elif value ==0:
            return 'red'
        elif value ==1:
            return 'orange'
        elif value == 2:
            return 'green'
        elif value == 3:
            return 'white'
    pixel = CircleMarker()
    pixel.location = (final_result.at[i, 'latitude'], final_result.at[i, 'longitude'])
    pixel.fill_color = assign_color(final_result.at[i, 'trend type'])
    pixel.stroke = False
    pixel.radius = 5
    pixel.fill_opacity = 1.0 

    m_trend.add_layer(pixel)
    
m_trend


Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…

<h3>Legend:</h3>
<ul>
<li>yellow - concealed trend
<li>red - no trend
<li>orange - linear trend
<li>green - quadratic trend
<li>white - cubic trend
    </ul>

<h2>Statistical significance graphic</h2>

In [14]:

m_significance = Map(
    center=AOI_zoom,
    zoom=12
)

# for i in range(0, 5):
for i in range(0, len(final_result)):
  
    pixel = CircleMarker()
    pixel.location = (final_result.at[i, 'latitude'], final_result.at[i, 'longitude'])
    pixel.color = assign_color(final_result.at[i, 'significance'])
    pixel.fill_color = assign_color(final_result.at[i, 'significance'])
    pixel.stroke = False
    pixel.radius = 5
    pixel.fill_opacity = 1.0 
    m_significance.add_layer(pixel)

m_significance

Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…

<h3>Legend</h3>
<ul>
    <li>yellow - statistically insignificant
        <li>orange - statistically significant
            </ul>

<h2>Polynomial degree graphic</h2>

In [15]:

m_degree = Map(
    center=AOI_zoom,
    zoom=10
)

# for i in range(0, 5):
for i in range(0, len(final_result)):
        
    pixel = CircleMarker()
    pixel.location = (final_result.at[i, 'latitude'], final_result.at[i, 'longitude'])
    pixel.fill_color = assign_color(final_result.at[i, 'degree'])
    pixel.stroke = False
    pixel.radius = 5
    pixel.fill_opacity = 1.0 
    m_degree.add_layer(pixel)


m_degree

Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…

<h3>Legend</h3>
<ul>
    <li>red - no trend</li>
    <li>orange - linear trend</li>
    <li>green - quadratic trend</li>
    <li>white - cubic trend</li>
        